In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [2]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [3]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [5]:
dataset = generate_dataset()

# save the dataset to a json file
with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [9]:
def run_prompt(test_case):
    """Merges the prompt and the test case inout, then returns the result"""
    prompt = f"""
Please solve the following task:
{test_case['task']}
"""
    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [ ]:
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)
    
    # TODO: implement grading logic
    score = 10  # placeholder

    return {
        "output": output,
        "test_case": test_case,
        "score": score
    }

In [11]:
def run_eval(dataset):
    """Loads the dataset and calls run_test_case for each test case"""
    results = []
    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)
    return results

In [12]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

In [13]:
print(json.dumps(results, indent=2))

[
  {
    "output": "# AWS S3 Bucket Name Validator\n\nHere's a comprehensive solution with validation and test cases:\n\n```python\nimport re\n\ndef validate_s3_bucket_name(bucket_name):\n    \"\"\"\n    Validates an AWS S3 bucket name according to AWS naming rules.\n    \n    Rules:\n    - Length: 3-63 characters\n    - Allowed characters: lowercase letters (a-z), numbers (0-9), and hyphens (-)\n    - Must start with a lowercase letter or number\n    - Must end with a lowercase letter or number\n    - Cannot contain consecutive hyphens\n    - Cannot be formatted as an IP address (e.g., 192.168.1.1)\n    \n    Args:\n        bucket_name (str): The bucket name to validate\n        \n    Returns:\n        tuple: (is_valid: bool, message: str)\n    \"\"\"\n    \n    # Check if input is a string\n    if not isinstance(bucket_name, str):\n        return False, \"Bucket name must be a string\"\n    \n    # Check length\n    if len(bucket_name) < 3:\n        return False, \"Bucket name must 